In [ ]:
%run business_search_data_load.py

In [ ]:
import pandas as pd
import numpy as np

if valid_data:
    restaurants = pd.DataFrame()
    for rest in valid_data:
        # flatten and append the new row to data frame
        restaurants = pd.concat([restaurants, pd.json_normalize(rest)])

    # drop duplicate restaurants
    restaurants = restaurants.drop_duplicates(subset = ['id'])

    # make a new row for each dictionary in the categories column
    rest_exp = restaurants.explode('categories').reset_index(drop=True)

    # encode all the information into new binary categorical columns 
    rest_enc = pd.get_dummies(rest_exp['categories'].apply(pd.Series))

    # concat the new columns to the exploded dataframe so that the rows match
    rest_final = pd.concat([rest_exp, rest_enc], axis=1)

    # change all column names to string and drop titles
    rest_final.columns = rest_final.columns.map(str)
    rest_final = rest_final.loc[:,~rest_final.columns.str.startswith('title')]

    # make rows unique and get the sum of categorical encoded cols grouping by business id 
    grouped = rest_final.groupby('id')
    for col in rest_final.columns[25:]:
        # col[:6] just drops the alias_ from the category name
        rest_final[col[6:]] = grouped[col].transform('sum')

    rest_final = rest_final.drop_duplicates(subset = ['id'])


    # repeat above steps to make a new row for each dictionary in the transaction col
    rest_exp = rest_final.explode('transactions').reset_index(drop=True)

    # encode all the information into new binary categorical columns 
    rest_enc = pd.get_dummies(rest_exp['transactions'].apply(pd.Series))

    # concat the new columns to the exploded dataframe so that the rows match
    rest_final = pd.concat([rest_exp, rest_enc], axis=1)

    grouped = rest_final.groupby('id')

    # make rows unique and get the sum of categorical encoded cols grouping by business id 
    for col in rest_final.columns[-3:]:
        rest_final[col[2:]] = grouped[col].transform('sum')

    rest_final = rest_final.drop_duplicates(subset = ['id'])


    # clean up
    rest_final = rest_final.loc[:,~rest_final.columns.str.startswith('alias')]
    rest_final = rest_final.loc[:,~rest_final.columns.str.startswith('0_')]
    rest_final = rest_final[rest_final['review_count'] > 0]
    rest_final = rest_final.drop(columns=['categories', 'location.state', 'location.country', 'location.display_address', 'transactions'])

In [ ]:
# feature engineering
if not rest_final.empty:

    # encode the neighborhoods
    rest_final = pd.concat([rest_final, pd.get_dummies(rest_final['neighborhood'])], axis=1)

    # make sure column names are strings
    rest_final.columns = rest_final.columns.map(str)

    # change all empty values to nan
    rest_final = rest_final.replace('', np.nan)

    # encode the price options to scale
    rest_final['price'].replace({'$':1, '$$':2, '$$$':3, '$$$$':4}, inplace=True)
    rest_final['price'].fillna(0, inplace=True)

    # has image from image_url
    rest_final['has_image'] = np.where(rest_final['image_url'].isna(), 0, 1)

    # has_phone from phone
    rest_final['has_phone'] = np.where(rest_final['phone'].isna(), 0, 1) 

    # has_st_add from location.address1
    rest_final['has_st_add'] = np.where(rest_final['location.address1'].isna(), 0, 1) 

    # for regression feature importance calculate a Balanced Rating Score (BRS)
    weight_average_rating = 0.7
    weight_review_count = 0.3

    # normalize review count using logarithm and min-max scaling
    rest_final['norm_count'] = np.log10(rest_final['review_count'] + 0.000000001)
    rest_final['norm_count'] = (rest_final['norm_count'] - rest_final['norm_count'].min()) / (rest_final['norm_count'].max() - rest_final['norm_count'].min())

    # calculate brs
    rest_final['brs'] = (weight_average_rating * (rest_final['rating'] / 5)) + (weight_review_count * rest_final['norm_count'])

    # cols to drop and rename
    rest_final = rest_final.drop(columns=['location.city', 'location.zip_code', 'image_url', 'is_closed', 'url', 'distance', 'norm_count', 'phone', 'display_phone', 'location.address1', 'location.address2', 'location.address3'])
    rest_final = rest_final.rename(columns={'coordinates.latitude': 'latitude', 'coordinates.longitude': 'longitude'})
    
    # check the integrity of the data
    print(f'Missing Values Detected: {rest_final.isnull().values.any()}')

In [ ]:
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
from shapely.geometry import Point

# create kde plot of restaurant location density
bos_map = gpd.read_file('Boston_Neighborhoods/Boston_Neighborhoods.shp')
geometry = [Point(xy) for xy in zip(rest_final['longitude'], rest_final['latitude'])]
crs = {'init':'epsg:4326'}
geo_df = gpd.GeoDataFrame(rest_final, # specify our data
                          crs=crs, # specify our coordinate reference system
                          geometry=geometry) # specify the geometry list we created
ax = geoplot.polyplot(bos_map, projection=gcrs.AlbersEqualArea(), zorder=2, figsize=(15, 15))
geoplot.kdeplot(geo_df, cmap='Reds', thresh=0, fill=True, clip=bos_map, ax=ax)

In [ ]:
# EDA
import matplotlib.pyplot as plt
import seaborn as sns

# distribution of balanced rating score
custom_bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
plt.hist(rest_final['brs'], bins=custom_bins, color='r', edgecolor='black', alpha=0.7)
median = rest_final['brs'].median()
mean = rest_final['brs'].mean()
plt.axvline(median, color='k', linestyle='dashed', linewidth=1, label='Median BRS')
plt.axvline(mean, color='k', linestyle='solid', linewidth=1, label='Mean BRS')
plt.xlabel('BRS')
plt.ylabel('Frequency')
plt.title('Distribution of BRS')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.show()

# distribution of review counts
plt.hist(rest_final['review_count'], color='b', edgecolor='black', alpha=0.7)
median = rest_final['review_count'].median()
mean = rest_final['review_count'].mean()
plt.axvline(median, color='k', linestyle='dashed', linewidth=1, label='Median Review Count')
plt.axvline(mean, color='k', linestyle='solid', linewidth=1, label='Mean Review Count')
plt.xlabel('Review Count')
plt.ylabel('Log Frequency')
plt.title('Log Distribution of Review Counts')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.25)
plt.yscale('log')
plt.show()

# distribution of ratings
rating_counts = rest_final.groupby(['rating'])['rating'].count()
rating_counts.plot(kind='barh', color='m')
plt.title('Number of Restaurants by Rating Category')
plt.xlabel('Count')
plt.ylabel('Ratings')
plt.grid(axis='x', linestyle='--', alpha=0.25)
plt.show()

# count of each neighborhood
custom_palette = sns.color_palette('Paired')
rating_counts = rest_final.groupby(['neighborhood'])['neighborhood'].count().sort_values(ascending=True)
rating_counts.plot(kind='barh', color=custom_palette)
plt.title('Number of Restaurants by Neighborhood')
plt.xlabel('Count')
plt.ylabel('Neighborhood')
plt.grid(axis='x', linestyle='--', alpha=0.25)
plt.show()

# counts of prices
custom_labels = ['No Price', '\$', '\$\$', '\$\$\$', '\$\$\$\$']
price_counts = rest_final.groupby(['price'])['price'].count()
price_counts.plot(kind='barh', color='g')
plt.title('Number of Restaurants by Price Range')
plt.xlabel('Count')
plt.ylabel('Prices')
plt.yticks(range(len(custom_labels)), custom_labels)
plt.grid(axis='x', linestyle='--', alpha=0.25)
plt.show()

# counts of tags
tag_counts = rest_final.loc[:, ['delivery', 'pickup', 'restaurant_reservation', 'has_image', 'has_phone', 'has_st_add']].sum().sort_values(ascending=True)
custom_palette = sns.color_palette('husl')
tag_counts.plot(kind='barh', color=custom_palette)
plt.axvline(x=rest_final.shape[0], color='k', linestyle='dashed', label=f'Total # of Restaurants') 
plt.legend(loc='lower right')
plt.title('Number of Restaurants by Tag Presence')
plt.xlabel('Count')
plt.ylabel('Tags')
plt.grid(axis='x', linestyle='--', alpha=0.25)
plt.show()

# cuisine occurences
cuisine_counts = rest_final.loc[:, 'acaibowls':'wraps'].sum().sort_values(ascending = True)
custom_palette = sns.color_palette('Paired')
cuisine_counts[-20:].plot(kind='barh', color=custom_palette)
plt.title('Top 20 Cuisines in Boston')
plt.xlabel('Count')
plt.ylabel('Cuisines')
plt.grid(axis='x', linestyle='--', alpha=0.25)
plt.show()

# median brs by neighborhood
neighborhoods = rest_final.groupby(['neighborhood'])['brs'].median().sort_values(ascending=True)
custom_palette = sns.color_palette('Paired')
neighborhoods.plot(kind='barh', color=custom_palette, label='')
plt.axvline(x=rest_final['brs'].median(), color='k', linestyle='dashed', label='Overall BRS Median')
plt.title('Median BRS by Neighborhood')
plt.xlabel('BRS')
plt.ylabel('Neighborhood')
plt.legend()
plt.grid(axis='x', linestyle='--', alpha=0.25)
plt.show()



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# splitting data into X (features) and y (target)
y = rest_final['brs']
neighborhoods = rest_final['neighborhood'].unique().tolist()
X = rest_final.drop((neighborhoods + ['id', 'name', 'review_count', 'rating', 'neighborhood', 'latitude', 'longitude', 'brs']), axis=1)

# splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
# number of features to consider at every split
max_features = ['auto', 'sqrt']
# maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 25]
# minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 10]
# method of selecting samples for training each tree
bootstrap = [True, False]
# create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# randomized search on random forest hyperparams
rs_forest = RandomForestRegressor()
forest_random = RandomizedSearchCV(estimator = rs_forest, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
forest_random.fit(X_train, y_train)
y_pred = forest_random.predict(X_test)

# calculate mean squared error and r2
rs_mse = mean_squared_error(y_test, y_pred)
rs_r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {rs_mse}')
print(f'R Squared Score is: {rs_r2}')
print(forest_random.best_params_)

In [ ]:
from sklearn.model_selection import GridSearchCV

# create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [74, 77, 80],
    'max_features': ['sqrt'],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [6, 7, 8, 9],
    'n_estimators': [100, 104, 105, 106, 107, 108, 109]
}

gs_forest = RandomForestRegressor()
# instantiate the grid search model
rf_grid_search = GridSearchCV(estimator = gs_forest, param_grid = param_grid,
                              scoring='neg_mean_squared_error', cv = 3, n_jobs = -1, verbose = 2)

# fit the grid search
rf_grid_search.fit(X_train, y_train)
y_pred = rf_grid_search.predict(X_test)
rf_best_params = rf_grid_search.best_params_
rf_best_estimator = rf_grid_search.best_estimator_

# calculate mean squared error and r2
rf_gs_mse = mean_squared_error(y_test, y_pred)
rf_gs_r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {rf_gs_mse}')
print(f'R Squared Score is: {rf_gs_r2}')
print('MSE Improvement of {:0.5f}%.'.format(100 * (rf_gs_mse - rs_mse) / rs_mse))
print('R2 Improvement of {:0.5f}%.'.format(100 * (rf_gs_r2 - rs_r2) / rs_r2))

# feature importance
gs_rf_importances = pd.DataFrame(rf_grid_search.best_estimator_.feature_importances_, index=X_train.columns, columns=['Importance'])
gs_rf_importances.sort_values(by='Importance', ascending=False, inplace=True)

In [ ]:
from xgboost import XGBRegressor

# baseline xgb regressor
baseline_xgb = XGBRegressor()
baseline_xgb.fit(X_train, y_train)
y_pred = baseline_xgb.predict(X_test)

# calculate mean squared error and r2
baseline_xgb_mse = mean_squared_error(y_test, y_pred)
baseline_xgb_r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {baseline_xgb_mse}')
print(f'R Squared Score is: {baseline_xgb_r2}')

In [ ]:
# grid search cv 
param_grid = {
    'n_estimators': [95, 100, 105],
    'max_depth': [2, 3, 4],
    'learning_rate': [0.12, 0.11, 0.13]
}

# create the GridSearchCV instance
xgb_grid_search = GridSearchCV(estimator=XGBRegressor(),
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',  # Use negative mean squared error for regression
                           cv=3) 

# fit the grid search 
xgb_grid_search.fit(X_train, y_train)
y_pred = xgb_grid_search.predict(X_test)
xgb_best_params = xgb_grid_search.best_params_
xgb_best_estimator = xgb_grid_search.best_estimator_

# calculate mean squared error and r2
xgb_gs_mse = mean_squared_error(y_test, y_pred)
xgb_gs_r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {xgb_gs_mse}')
print(f'R Squared Score is: {xgb_gs_r2}')
print('MSE Improvement of {:0.5f}%.'.format(100 * (xgb_gs_mse - baseline_xgb_mse) / baseline_xgb_mse))
print('R2 Improvement of {:0.5f}%.'.format(100 * (xgb_gs_r2 - baseline_xgb_r2) / baseline_xgb_r2))

# feature importance
gs_xgb_importances = pd.DataFrame(xgb_best_estimator.feature_importances_, index=X_train.columns, columns=['Importance'])
gs_xgb_importances.sort_values(by='Importance', ascending=False, inplace=True)

In [ ]:
# let's compare the metrics and feature importances of the two tuned models
print('MSE Difference of {:0.5f}%.'.format(xgb_gs_mse - rf_gs_mse))
print('R2 Difference of {:0.5f}%.'.format(xgb_gs_r2 - rf_gs_r2))

# feature selection
X_rf = X[gs_rf_importances.index[:20].tolist()]
X_xgb = X[gs_xgb_importances.index[:20].tolist()]

In [ ]:
# retune after feature selection
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y, test_size=0.25, random_state=42)

# create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [20, 50, 80],
    'max_features': ['auto'],
    'min_samples_leaf': [2, 5, 10],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [50, 100, 150]
}

# create a based model
gs_forest = RandomForestRegressor()
# instantiate the grid search model
rf_grid_search = GridSearchCV(estimator = gs_forest, param_grid = param_grid,
                            scoring='neg_mean_squared_error', cv = 3, n_jobs = -1, verbose = 2)

# fit the grid search
rf_grid_search.fit(X_train_rf, y_train_rf)
y_pred_rf = rf_grid_search.predict(X_test_rf)
rf_best_params = rf_grid_search.best_params_
rf_best_estimator = rf_grid_search.best_estimator_

# calculate mean squared error and r2
rf_gs_mse = mean_squared_error(y_test_rf, y_pred_rf)
rf_gs_r2 = r2_score(y_test_rf, y_pred_rf)

# feature importance
gs_rf_importances = pd.DataFrame(rf_grid_search.best_estimator_.feature_importances_, index=X_train_rf.columns, columns=['Importance'])
gs_rf_importances.sort_values(by='Importance', ascending=False, inplace=True)

In [ ]:
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X_xgb, y, test_size=0.25, random_state=42)

# grid search cv 
param_grid = {
    'n_estimators': [50, 75, 100, 110],
    'max_depth': [2, 3, 4, 6, 8, 12],
    'learning_rate': [0.25, 0.2, 0.15, 0.12, 0.11, 0.13]
}

# create the GridSearchCV instance
xgb_grid_search = GridSearchCV(estimator=XGBRegressor(),
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',  # Use negative mean squared error for regression
                           cv=3) 

# fit the grid search 
xgb_grid_search.fit(X_train_xgb, y_train_xgb)
xgb_best_estimator = xgb_grid_search.best_estimator_
y_pred_xgb = xgb_best_estimator.predict(X_test_xgb)
xgb_best_params = xgb_grid_search.best_params_

# calculate mean squared error and r2
xgb_gs_mse = mean_squared_error(y_test_xgb, y_pred_xgb)
xgb_gs_r2 = r2_score(y_test_xgb, y_pred_xgb)


# feature importance
gs_xgb_importances = pd.DataFrame(xgb_best_estimator.feature_importances_, index=X_train_xgb.columns, columns=['Importance'])
gs_xgb_importances.sort_values(by='Importance', ascending=False, inplace=True)


In [ ]:
# compare the metrics and feature importances of the two tuned models
print(f'Random Forest Mean Squared Error: {rf_gs_mse}')
print(f'Random Forest R Squared Score is: {rf_gs_r2}')

print(f'XGBoost Mean Squared Error: {xgb_gs_mse}')
print(f'XGBoost R Squared Score is: {xgb_gs_r2}')

print('MSE Difference (XGBoost - Random Forest) of {:0.5f}%.'.format(xgb_gs_mse - rf_gs_mse))
print('R2 Difference (XGBoost - Random Forest) of {:0.5f}%.'.format(xgb_gs_r2 - rf_gs_r2))

# random forest performs a little better
plt.scatter(y_test_rf, y_pred_rf, c='red', alpha=0.7, label='Restaurants')
fit = np.polyfit(y_test_rf, y_pred_rf, 1)
plt.plot(y_test_rf, fit[0] * y_test_rf + fit[1], color='k', linestyle='solid', label='Trendline')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Scatter Plot of Actual vs. Predicted Values')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.25)
plt.show()

residuals = [y_test - y_pred for y_test, y_pred in zip(y_test_rf, y_pred_rf)]
sns.displot(residuals, kde=True, color='red', bins=15, edgecolor='black')
plt.xlabel('Residuals (Actual - Predicted)')
plt.ylabel('Density')
plt.title('Distribution of Residuals')
mean_residual = round(np.mean(residuals), 5)
median_residual = round(np.median(residuals), 5)
std_dev = round(np.std(residuals), 5)
plt.text(0.45, 0.1, f'Mean: {mean_residual}\nMedian: {median_residual}\nStd Dev: {std_dev}', transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.8))
plt.grid(True, linestyle='--', alpha=0.25)
plt.show()

In [ ]:
# random forest feature importance
plt.figure(figsize=(10,8))
sns.barplot(x=gs_rf_importances['Importance'], y=gs_rf_importances.index)
plt.title('Random Forest Feature Importance')
plt.xlabel('Feature Importance')
plt.ylabel('Feature Names')
plt.grid(axis='x', linestyle='--', alpha=0.25)
plt.show()

# xgboost feature importance
plt.figure(figsize=(10,8))
sns.barplot(x=gs_xgb_importances['Importance'], y=gs_xgb_importances.index)
plt.title('XGBoost Feature Importance')
plt.xlabel('Feature Importance')
plt.ylabel('Feature Names')
plt.grid(axis='x', linestyle='--', alpha=0.25)
plt.show()

In [ ]:
# plot the brs by important features
custom_labels = ['No Price', '\$', '\$\$', '\$\$\$', '\$\$\$\$']
colors = sns.color_palette('Set2')
sns.boxplot(x=X_rf['price'], y=y, palette=colors)
plt.title('Distribution of Balanced Rating Score by Price Range')
plt.xlabel('Price Range')
plt.ylabel('BRS')
plt.xticks(range(len(custom_labels)), custom_labels)
plt.grid(axis='y', linestyle='--', alpha=0.25)
plt.show()

feat_graph_list = list(X_rf.columns.values)
feat_graph_list.remove('price')
for feature in feat_graph_list:
    custom_labels = ['No', 'Yes']
    colors = sns.color_palette('Set2')
    sns.boxplot(x = X[feature], y = y, palette=colors)
    plt.title(f'Distribution of Balanced Rating Score by {feature}')
    plt.xlabel(feature)
    plt.ylabel('BRS')
    plt.xticks(range(len(custom_labels)), custom_labels)
    plt.grid(axis='y', linestyle='--', alpha=0.25)
    plt.show()